### Preprocess iEEG data

In [3]:
import numpy as np
import mne
import pandas as pd
import mne_bids

from utils import resample, smooth_signal

# Read iEEG

### Specify subject's directory and meta-information 

In [4]:
bids_dir = '/media/snail-fuji/Nowy/ieeg/movies/ds003688-download/'

In [5]:
subjects = mne_bids.get_entity_vals(bids_dir, 'subject')

In [6]:
subject = '01'
acquisition = 'clinical'
task = 'film'
datatype = 'ieeg'
session = 'iemu'

### Load channels

In [7]:
channels_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='channels',
                                    extension='tsv',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)
channels = pd.read_csv(str(channels_path.match()[0]), sep='\t', header=0, index_col=None)

/tmp/ipykernel_12579/3115014924.py:1: FutureWarning: extension should start with a period ".", but got: "tsv". Prepending "." to form: ".tsv". This will raise an exception starting with MNE-BIDS 0.12.
  channels_path = mne_bids.BIDSPath(subject=subject,


### Load data information

In [8]:
data_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='ieeg',
                                    extension='vhdr',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)
raw = mne.io.read_raw_brainvision(str(data_path.match()[0]), scale=1.0, preload=False, verbose=True)
raw.set_channel_types({ch_name: str(x).lower()
                if str(x).lower() in ['ecog', 'seeg', 'eeg'] else 'misc'
                                for ch_name, x in zip(raw.ch_names, channels['type'].values)})
raw.drop_channels([raw.ch_names[i] for i, j in enumerate(raw.get_channel_types()) if j == 'misc'])

/tmp/ipykernel_12579/2865297635.py:1: FutureWarning: extension should start with a period ".", but got: "vhdr". Prepending "." to form: ".vhdr". This will raise an exception starting with MNE-BIDS 0.12.
  data_path = mne_bids.BIDSPath(subject=subject,


Extracting parameters from /media/snail-fuji/Nowy/ieeg/movies/ds003688-download/sub-01/ses-iemu/ieeg/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vhdr...
Setting channel info structure...


/tmp/ipykernel_12579/2865297635.py:10: RuntimeWarning: The unit for channel(s) AH+, ECG+, EMG+, MKR1+, MKR2+, ORB+, abdo+, thor+ has changed from V to NA.
  raw.set_channel_types({ch_name: str(x).lower()


Measurement date,"January 01, 1900 00:00:00 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,103 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,1024.00 Hz


### Discard bad channels

In [9]:
bad_channels = channels['name'][(channels['type'].isin(['ECOG', 'SEEG'])) & (channels['status'] == 'bad')].tolist()
raw.info['bads'].extend([ch for ch in bad_channels])
raw.drop_channels(raw.info['bads'])

Measurement date,"January 01, 1900 00:00:00 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,101 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,1024.00 Hz


### Load raw data

In [10]:
raw.load_data()

Reading 0 ... 860253  =      0.000 ...   420.045 secs...


Measurement date,"January 01, 1900 00:00:00 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,101 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,1024.00 Hz


### Apply notch filter to remove line noise

In [11]:
raw.notch_filter(freqs=np.arange(50, 251, 50))

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13517 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    5.3s


Measurement date,"January 01, 1900 00:00:00 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,101 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,1024.00 Hz


### Apply common average reference to remove common noise and trends

In [12]:
raw_car, _ = mne.set_eeg_reference(raw.copy(), 'average')

sEEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('sEEG',) reference.


### Extract signal in gamma range, use Hilbert transform, but can also play around with wavelet decomposition options

In [13]:
gamma = raw_car.copy().filter(60, 120).apply_hilbert(envelope=True).get_data().T

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 60 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 60.00
- Lower transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 52.50 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 451 samples (0.220 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    4.0s


In [14]:
# temp = mne.time_frequency.tfr_array_morlet(np.expand_dims(self.raw_car.copy()._data, 0), # (n_epochs, n_channels, n_times)
#                                                      sfreq=self.raw.info['sfreq'],
#                                                      freqs=np.arange(60, 120),
#                                                      verbose=True,
#                                                      n_cycles=4.,
#                                                      n_jobs=1)
# gamma = np.mean(np.abs(temp), 2).squeeze().T

### Read annotation with event markers

In [15]:
# custom_mapping = {'Stimulus/music': 2, 'Stimulus/speech': 1,
#                   'Stimulus/end task': 5}  # 'Stimulus/task end' in laan
# events, event_id = mne.events_from_annotations(raw_car, event_id=custom_mapping,
#                                                          use_rounding=False)

# Read annotations

Задача - прочитать больше аннотаций

In [16]:
import os

In [17]:
annotations_path = '/media/snail-fuji/Nowy/ieeg/movies/ds003688-download/stimuli/annotations/video'

In [18]:
file_paths = os.listdir(annotations_path)

In [19]:
file_paths[0]

'video_annotation_acrobat.tsv'

In [20]:
all_annotations = []

for file_path in file_paths:
    file_annotation_path = annotations_path + '/' + file_path
    annotations_df = pd.read_csv(file_annotation_path, sep='\t')
    annotations_df['label'] = file_path.replace('video_annotation_', '').split('.')[0]
    all_annotations.append(annotations_df)

In [21]:
all_annotations_df = pd.concat(all_annotations)

In [22]:
all_annotations_df.to_csv('anchors.csv', index=False)

# Assign new annotations

In [23]:
sfreq = raw_car.info['sfreq']

In [24]:
all_annotations_df['onset_sample'] = all_annotations_df['onset'] * sfreq

In [25]:
all_annotations_df['onset'].max()

386.76

In [26]:
raw_car.times.max()

420.04541015625

In [27]:
all_annotations_df['label'].value_counts().iloc[0:10]

label
pippi         26
girl          23
woman         23
smile         23
hair          22
head          22
man           22
seated        21
young         19
adolescent    19
Name: count, dtype: int64

In [28]:
all_annotations_labels = all_annotations_df['label'].value_counts().iloc[0:10]

In [29]:
all_annotations_df['event1'] = 0
all_annotations_df['event_id'] = all_annotations_df['label'].apply(lambda x: x in all_annotations_labels).astype(int)

In [30]:
all_annotations_labels_dict = {
    'target': 1,
    'other': 0
}

In [31]:
all_annotations_df['event_id'].value_counts()

event_id
0    836
1    220
Name: count, dtype: int64

In [32]:
events = all_annotations_df[['onset_sample', 'event1', 'event_id']].fillna(len(all_annotations_labels_dict)).drop_duplicates(subset='onset_sample').astype(int).values

In [33]:
# used_annotations = set(events.T[-1])

In [34]:
all_annotations_labels_dict_filtered = all_annotations_labels_dict.copy()

In [35]:
epochs = mne.Epochs(
    raw_car, 
    events, 
    event_id=all_annotations_labels_dict_filtered, 
    tmin=-5, 
    tmax=0,
    baseline=(None, 0),
    preload=True
)

Not setting metadata
165 matching events found
Setting baseline interval to [-5.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 165 events and 10241 original time points ...


/tmp/ipykernel_12579/2756416248.py:1: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(


4 bad epochs dropped


In [36]:
epochs = epochs.decimate(5)

/tmp/ipykernel_12579/1337902028.py:1: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=5 parameter will result in a sampling frequency of 409.6 Hz, which can cause aliasing artifacts.
  epochs = epochs.decimate(5)


In [37]:
targets = epochs.events.T[-1]

In [38]:
len(targets)

161

In [39]:
# raw_car.plot(
#     events=events, 
#     start=0, 
#     duration=180, 
#     color='gray', 
#     event_color={2: 'g', 1: 'r'}, 
#     bgcolor='w'
# )

# Building a model

In [40]:
# ! pip install braindecode

In [41]:
from braindecode.models.util import models_dict

print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [42]:
from braindecode.models import ShallowFBCSPNet

In [43]:
epochs.times.shape

(2049,)

In [60]:
# model = ShallowFBCSPNet(
#     n_chans=len(epochs.ch_names),
#     n_times=epochs.times.shape[0],
#     n_outputs=len(set(targets)),
#     final_conv_length='auto',
# )
# print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 101, 2049]            [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 101, 2049]            [1, 101, 2049, 1]         --                        --
├─Rearrange (dimshuffle): 1-2            [1, 101, 2049, 1]         [1, 1, 2049, 101]         --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 2049, 101]         [1, 40, 2025, 1]          162,640                   --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 2025, 1]          [1, 40, 2025, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 2025, 1]          [1, 40, 2025, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 2025, 1]          [1, 40, 131, 1]           --                        [75, 1]
├─Express

In [72]:
n_epochs = 100
lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64

In [73]:
from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier
import torch

In [74]:
net = EEGClassifier(
    'ShallowFBCSPNet',
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.AdamW,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    module__final_conv_length='auto',
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)

In [75]:
np.mean(targets)

0.20496894409937888

In [76]:
net.fit(epochs, targets, epochs=n_epochs)

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  -------
      1            0.2266        1.0661       0.1212            0.1212        6.5202  0.0006  14.1447
      2            0.2812        0.5381       0.1515            0.1515        4.9241  0.0006  13.0210
      3            0.4922        0.2780       0.0909            0.0909        2.9957  0.0006  12.7258
      4            0.6797        0.1295       0.2121            0.2121        2.2876  0.0006  12.5576
      5            0.7812        0.1546       0.2121            0.2121        2.1303  0.0006  12.4831
      6            0.8438        0.1011       0.2121            0.2121        2.0011  0.0006  12.3457
      7            0.9141        0.1060       0.2424            0.2424        1.9077  0.0006  12.3075
      8            0.9531        0.0547       0.3030            0.3030        1.68

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 101, 2049]            [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 101, 2049]            [1, 101, 2049, 1]         --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 101, 2049, 1]         [1, 1, 2049, 101]         --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 2049, 101]         [1, 40, 2025, 1]          162,640                   --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 2025, 1]          [1, 40, 2025, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

Что нужно для того, чтобы скачать датасет на гугл диск?

In [49]:
net.predict(epochs)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1])

In [50]:
net.predict_proba(epochs)

array([[8.90774727e-01, 1.09225281e-01],
       [7.70938933e-01, 2.29061037e-01],
       [1.81875035e-01, 8.18124950e-01],
       [9.87322271e-01, 1.26777468e-02],
       [9.12390351e-01, 8.76096264e-02],
       [5.06068647e-01, 4.93931383e-01],
       [8.84952009e-01, 1.15047991e-01],
       [6.03151560e-01, 3.96848410e-01],
       [8.95363152e-01, 1.04636885e-01],
       [2.68654502e-03, 9.97313440e-01],
       [1.46892788e-02, 9.85310674e-01],
       [2.17545368e-02, 9.78245437e-01],
       [3.76476976e-03, 9.96235192e-01],
       [2.38898490e-03, 9.97611046e-01],
       [1.30839879e-03, 9.98691618e-01],
       [4.07257557e-01, 5.92742443e-01],
       [4.59133042e-03, 9.95408595e-01],
       [5.45587800e-02, 9.45441246e-01],
       [1.26477228e-02, 9.87352252e-01],
       [1.29940445e-02, 9.87005949e-01],
       [2.18394957e-02, 9.78160560e-01],
       [8.45161378e-01, 1.54838637e-01],
       [4.12051333e-03, 9.95879531e-01],
       [4.81703691e-03, 9.95182931e-01],
       [9.374397

In [ ]:
del net